In [1]:
import os
from datasets import load_dataset, concatenate_datasets, DownloadMode
import re
import datetime
from concurrent.futures import ThreadPoolExecutor


/home/senya/.cache/pypoetry/virtualenvs/drip-search-J_7n9XPv-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ROOT_DIR = '/mnt/e/code/politech/nlp/drip-search'
data_dir = os.path.join(ROOT_DIR, 'data')
subsets = ["20231101.uk", "20231101.en"]
ds = concatenate_datasets([load_dataset("wikimedia/wikipedia", subset, cache_dir=data_dir, download_mode=DownloadMode.REUSE_DATASET_IF_EXISTS)['train'] for subset in subsets ])

In [3]:
drip_titles = ['cp company', 'stone island', 'nike', 'arcteryx', 'adidas', 'timberland', 'chrome hearts']

In [ ]:
import re
from concurrent.futures import ThreadPoolExecutor

def like_pattern(value, pattern):
    regex = pattern.replace('%', '.*').replace('_', '.')
    return re.fullmatch(regex, value) is not None

def check_pattern_for_doc(doc, drip_titles):
    return any([like_pattern(doc['title'].lower(), f'%{drip_title}% ') for drip_title in drip_titles])

# Parallelize the filtering of documents using ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    # Map the function across the documents
    drip_docs = list(executor.map(lambda doc: check_pattern_for_doc(doc, drip_titles), ds))

drip_docs = [doc for doc, match in zip(ds, drip_docs) if match]

print([drip_doc['title'] for drip_doc in drip_docs])

In [ ]:
for i, doc in enumerate(drip_docs):
    filename = f'{i}-{doc['title'].lower().replace('/', '')}-{datetime.datetime.now().isoformat()}.txt'
    filepath = os.path.join(data_dir, filename) 
    with open(filepath, 'x') as f:
        f.write(doc['text'])